In [ ]:
!pip install openai

In [ ]:
import os
import openai
import json

In [ ]:
from openai import OpenAI

api_key  = 'Your-API-Key'

client = OpenAI(api_key=api_key)


## Example 1 - Simple chat example

In [ ]:
# Say hello and get a response

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message.content)

## Function Calling Example

In [ ]:
# Define a dummy function

def get_product_of_two_numbers(num1, num2):
    
    answer = str(num1 * num2)
    
    return answer

In [ ]:
tools = [
  {
      
    "type": "function", # What kind of tool is it
    "function": {
      "name": "get_product_of_two_numbers",
      "description": "Mutiplies two numbers and returns an answer.",
        # What do we want chatgpt to return
      "parameters": {
        "type": "object", # We want an object
          
        "properties": {
          "first_number": {
            "type": "integer",
            "description": "The first number e.g. 3",
          },
          "second_number": {
            "type": "integer",
            "description": "The first number e.g. 5",
          }   
        }, # close properties
          
        # Both numbers are required
        "required": ["first_number", "second_number"],
          
      } # close parameters
          
    } # close function
        
  } # close first curly brace
]

In [ ]:
# Send a message to chatgpt

messages = [{"role": "user", "content": "What is the product of 5 and 2?"}]
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  tools=tools,
  tool_choice="auto"
)

In [ ]:
print(completion)

In [ ]:
# Get the reason for the finish
completion.choices[0].finish_reason

In [ ]:
# Get the function name
# Note the this is important because there could be many functions
function_name = completion.choices[0].message.tool_calls[0].function.name

function_name

In [ ]:
completion.choices[0].message.tool_calls[0].function.arguments

In [ ]:
arguments = completion.choices[0].message.tool_calls[0].function.arguments

arguments

In [ ]:
# Convert to json format

json_arguments = json.loads(arguments)

json_arguments 

In [ ]:
# Get the arguments that chatgpt has provided 

print(json_arguments['first_number'])
print(json_arguments['second_number'])

In [ ]:
# Call the function

num1 = json_arguments['first_number']
num2 = json_arguments['second_number']

answer = get_product_of_two_numbers(num1, num2)

answer

In [ ]:
# We add a new messages to the messages list and send it
# to chatgpt

messages = [{"role": "user", "content": "What is the product of 5 and 2?"},
           {'role': 'function', 'name': function_name, 'content': answer}]

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  tools=tools,
  tool_choice="auto"
)

In [ ]:
completion

In [ ]:
# This is the final response from Chatgpt that we show to the user

completion.choices[0].message.content

## Example 2: Retrieving student info

In [ ]:
import json

# Your JSON object
json_data = '''
{
  "students": [
    {
      "full_name": "John Smith",
      "student_id": "A123456",
      "date_of_birth": "1998-05-15"
    },
    {
      "full_name": "Emily Johnson",
      "student_id": "B789012",
      "date_of_birth": "2000-09-22"
    },
    {
      "full_name": "Michael Davis",
      "student_id": "C345678",
      "date_of_birth": "1999-03-08"
    },
    {
      "full_name": "Sophia Garcia",
      "student_id": "D901234",
      "date_of_birth": "2001-11-10"
    },
    {
      "full_name": "Ethan Wilson",
      "student_id": "E567890",
      "date_of_birth": "1997-07-04"
    }
  ]
}
'''

# Parse the JSON data
data = json.loads(json_data)

def get_student_info(student_id):
    for student in data["students"]:
        if student["student_id"] == student_id:
            return student
    return None  # Return None if the student_id is not found



In [ ]:
# Example: Get info for student with student_id "C345678"
student_id_to_search = "C345678"
student_info = get_student_info(student_id_to_search)

if student_info:
    print(f"Student Info for ID {student_id_to_search}: {student_info}")
else:
    print(f"Student with ID {student_id_to_search} not found.")


In [ ]:
tools = [
  {
      
    "type": "function", # What kind of tool is it
    "function": {
      "name": "get_student_info",
      "description": "Given a student id, this function returns a dictionary containing the full name and date of birth of a student.",
        # What do we want chatgpt to return
      "parameters": {
        "type": "object", # We want an object
          
        "properties": {
          "student_id": {
            "type": "string",
            "description": "The id of the student.",
          }
        }, # close properties
          
        # Both numbers are required
        "required": ["student_id"],
          
      } # close parameters
          
    } # close function
        
  } # close first curly brace
]

In [ ]:
# Send a message to chatgpt

messages = [{"role": "user", "content": "What is the full name of the student with id A123456?"}]
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  tools=tools,
  tool_choice="auto"
)

arguments = completion.choices[0].message.tool_calls[0].function.arguments

arguments

In [ ]:
# Convert to json format

json_arguments = json.loads(arguments)

json_arguments 

In [ ]:
student_id = json_arguments['student_id']

student_info = get_student_info(student_id)

student_info = str(student_info)

student_info

In [ ]:
# Get the function name
# Note the this is important because there could be many functions
function_name = completion.choices[0].message.tool_calls[0].function.name

function_name

In [ ]:
# We add a new messages to the messages list and send it
# to chatgpt

#answer = student_info['full_name']

messages = [{"role": "user", "content": "What is the full name of the student with id A123456?"},
           {'role': 'function', 'name': function_name, 'content': student_info}]

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  tools=tools,
  tool_choice="auto"
)


# This is the final response from Chatgpt that we show to the user

answer = completion.choices[0].message.content

answer

In [ ]:
messages = [{"role": "user", "content": "What is the full name of the student with id A123456?"},
           {'role': 'function', 'name': function_name, 'content': student_info},
           {'role': 'assistant', 'content': answer},
           {"role": "user", "content": "What is his date of birth?"}]

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  tools=tools,
  tool_choice="auto"
)
      
        
completion.choices[0].message.content